In [1]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/jovyan/work/phd/datasets/gtex/10/random_selection")

In [2]:
df=pd.read_csv("mainTable.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, ENSG00000271614 to ENSG00000253951
Columns: 1000 entries, GTEX-111CU-1426-SM-5GZYP to GTEX-ZZPU-0926-SM-5GZYT
dtypes: float64(1000)
memory usage: 22.9+ MB


In [3]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g[:15])
            file.write(" ")
        file.write("\n")

In [4]:
gc.collect()

22

In [5]:
os.system("export PATH=$PATH:/home/jovyan/work/phd/topicmapping/bin")
os.system("topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm")

32512

In [6]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
938,ENSG00000171431,0,84
2330,ENSG00000131981,1,1000
2165,ENSG00000143416,2,981
1721,ENSG00000132465,3,612
2002,ENSG00000121900,4,916
...,...,...,...
126,ENSG00000225404,2551,1
127,ENSG00000253319,2552,1
128,ENSG00000234455,2553,1
129,ENSG00000259791,2554,1


In [7]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2,Topic 3
i_doc,,,,
0,GTEX-111CU-1426-SM-5GZYP,215.075000,0.077544,785.170000
1,GTEX-111FC-0226-SM-5N9B8,0.082094,64.963100,935.277000
2,GTEX-111YS-0006-SM-5NQBE,1000.080000,0.077544,0.162203
3,GTEX-111YS-0326-SM-5GZZ3,654.208000,88.135600,257.979000
4,GTEX-111YS-2426-SM-5GZZQ,0.082094,0.077544,1000.160000
...,...,...,...,...
995,GTEX-ZZ64-0826-SM-5E449,0.082097,539.855000,460.385000
996,GTEX-ZZPT-3026-SM-5GZXH,156.992000,843.167000,0.162203
997,GTEX-ZZPU-0226-SM-5N9BV,0.082094,0.077544,1000.160000


In [8]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [9]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [10]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [11]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)